In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
# Read the csv file
df = pd.read_csv('data.csv', header=None)

# Remove empty rows
df = df.replace(np.nan,0)

# Convert the dataframe to a numpy array
data = df.to_numpy()

In [2]:
data

array([[0.0, 'R', 'P', ..., 'R', 'R', 'R']], dtype=object)

In [3]:
data.shape

(1, 200000)

In [4]:
Mapping = {'R':1,'P':2,'S':3}

def map_values(x):
    return Mapping.get(x, x)

In [5]:
vectorized_map_values = np.vectorize(map_values)
converted_data = vectorized_map_values(data)

In [6]:
converted_data.dtype

dtype('float64')

In [7]:
segments = []

for i in range(0, converted_data.shape[1] - 20):
    segment = converted_data[0][i:i+20]
    segments.append(segment)


In [8]:
segments

[array([0., 1., 2., 2., 3., 1., 1., 2., 2., 3., 1., 1., 2., 2., 3., 1., 1.,
        2., 2., 3.]),
 array([1., 2., 2., 3., 1., 1., 2., 2., 3., 1., 1., 2., 2., 3., 1., 1., 2.,
        2., 3., 1.]),
 array([2., 2., 3., 1., 1., 2., 2., 3., 1., 1., 2., 2., 3., 1., 1., 2., 2.,
        3., 1., 1.]),
 array([2., 3., 1., 1., 2., 2., 3., 1., 1., 2., 2., 3., 1., 1., 2., 2., 3.,
        1., 1., 2.]),
 array([3., 1., 1., 2., 2., 3., 1., 1., 2., 2., 3., 1., 1., 2., 2., 3., 1.,
        1., 2., 2.]),
 array([1., 1., 2., 2., 3., 1., 1., 2., 2., 3., 1., 1., 2., 2., 3., 1., 1.,
        2., 2., 3.]),
 array([1., 2., 2., 3., 1., 1., 2., 2., 3., 1., 1., 2., 2., 3., 1., 1., 2.,
        2., 3., 1.]),
 array([2., 2., 3., 1., 1., 2., 2., 3., 1., 1., 2., 2., 3., 1., 1., 2., 2.,
        3., 1., 1.]),
 array([2., 3., 1., 1., 2., 2., 3., 1., 1., 2., 2., 3., 1., 1., 2., 2., 3.,
        1., 1., 2.]),
 array([3., 1., 1., 2., 2., 3., 1., 1., 2., 2., 3., 1., 1., 2., 2., 3., 1.,
        1., 2., 2.]),
 array([1., 1., 2., 

In [9]:
dataset = tf.data.Dataset.from_tensor_slices(converted_data)

<TensorSliceDataset element_spec=TensorSpec(shape=(200000,), dtype=tf.float64, name=None)>

In [22]:
BATCH_SIZE = 20
batched_data= dataset.batch(BATCH_SIZE+1, drop_remainder=True)

In [23]:
def split_input_target(chunk):  
    input_text = chunk[:-1]  
    target_text = chunk[1:]  
    return input_text, target_text

dataset = batched_data.map(split_input_target)

TypeError: in user code:


    TypeError: outer_factory.<locals>.inner_factory.<locals>.tf__split_input_target() takes 1 positional argument but 2 were given


In [ ]:
dataset

In [12]:
batched_data


<BatchDataset element_spec=TensorSpec(shape=(20, 200000), dtype=tf.float64, name=None)>

In [13]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
        tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

VOCAB_SIZE = 4
EMBEDDING_DIM = 32
RNN_UNITS = 1024

In [15]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (20, None, 32)            128       
                                                                 
 lstm (LSTM)                 (20, None, 1024)          4329472   
                                                                 
 dense (Dense)               (20, None, 4)             4100      
                                                                 
Total params: 4,333,700
Trainable params: 4,333,700
Non-trainable params: 0
_________________________________________________________________


In [19]:
model.fit(batched_data,epochs = 10) 
#next attempt check what datasets can go into this function, make sure I am puttin the right stuff

Epoch 1/10


ValueError: Unexpected result of `train_function` (Empty logs). Please use `Model.compile(..., run_eagerly=True)`, or `tf.config.run_functions_eagerly(True)` for more information of where went wrong, or file a issue/bug to `tf.keras`.

In [ ]:
model.input_shape